In [2]:
import numpy as np
import networkx as nx
import random

In [4]:
class Graph():
    def __init__(self, nx_S, nx_A, is_directed_struc, is_directed_attr, p_struc, q_struc, p_attr, q_attr):
        self.G = [nx_S, nx_A]
        self.is_directed = [is_directed_struc, is_directed_attr]
        self.p = [p_struc, p_attr]
        self.q = [q_struc, q_attr]
        self.alias_nodes = [None, None]
        self.alias_edges = [None, None]
        self.trans_weight = [None, None]
        self.ct = [0, 0]

    # Modified node2vec walk for multi-layered graph with structure and content
    def node2vec_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]
        while len(walk) < walk_length:
            cur = walk[-1]
            # Decide the layer structure/attribute for the current node
            # taking upper as structure and lower level as attribute
            up = self.trans_weight[1][cur]
            down = self.trans_weight[0][cur]
            pu = up / (up + down)  # probability to select in structure layer
            pd = 1 - pu  # probability to select in attribute layer

            x = random.random()  # random num between 0---1
            if x < pu:  # if pu is large then more chances of Reference being selected
                ind = 0
            else:
                ind = 1
            self.ct[ind] += 1  # to get count which layer the Random walk is in.

            cur_nbrs = sorted(self.G[ind].neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_draw(alias_nodes[ind][cur][0], alias_nodes[ind][cur][1])])
                else:
                    prev = walk[-2]
                    if (prev, cur) not in alias_edges[ind]:  # when the edge is not in other graph
                        walk.append(cur_nbrs[alias_draw(alias_nodes[ind][cur][0], alias_nodes[ind][cur][1])])
                    else:
                        e1 = alias_edges[ind][(prev, cur)][0]
                        e2 = alias_edges[ind][(prev, cur)][1]
                        tmp = alias_draw(e1, e2)
                        next = cur_nbrs[tmp]
                        walk.append(next)
            else:
                break

        return walk
    def simulate_walks(self, num_walks, walk_length):
        G = self.G[0]  # we can take any graph as we just need to find the nodes
        walks = []
        nodes = list(G.nodes())
        for walk_iter in range(num_walks):
            print('Walk iteration :: %s / %s' % (str(walk_iter + 1), str(num_walks)))
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))
            print('Walk count in Structure layer :: %s  , Attribute layer :: %s' % (str(self.ct[0]), str(self.ct[1])))
            self.ct = [0, 0]
        return walks

    def get_level_transition_weight(self, ind):
        G = self.G[ind]
        mat = nx.to_scipy_sparse_matrix(G)
        if ind == 0:
            avg = 1.0
        else:
            avg = 1.0 * np.sum(mat) / G.number_of_edges()
        if ind == 0:
            print('Threshold for structure layer :: %s' % str(avg))
        else:
            print('Threshold for attribute layer :: %s' % str(avg))
        mat = mat >= avg
        tau = np.sum(mat, axis=1)
        self.trans_weight[ind] = np.log(np.e + tau)

    def get_alias_edge(self, src, dst, ind):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G[ind]
        p = self.p[ind]
        q = self.q[ind]

        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            if dst_nbr == src:
                unnormalized_probs.append(G[dst][dst_nbr]['weight'] / p)
            elif G.has_edge(dst_nbr, src):
                unnormalized_probs.append(G[dst][dst_nbr]['weight'])
            else:
                unnormalized_probs.append(G[dst][dst_nbr]['weight'] / q)
        norm_const = sum(unnormalized_probs)
        normalized_probs = [float(u_prob) / norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self, ind):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G[ind]
        is_directed = self.is_directed[ind]

        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs = [float(u_prob) / norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)

        alias_edges = {}
        triads = {}

        if is_directed:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1], ind)
        else:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1], ind)
                alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0], ind)

        self.alias_nodes[ind] = alias_nodes
        self.alias_edges[ind] = alias_edges

        return


In [5]:
def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

In [6]:
def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)
    kk = int(np.floor(np.random.rand() * K))
    tmp = q[kk]
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [7]:
def parse_args():
    '''
    Parses the MIRand arguments.
    '''
    parser = argparse.ArgumentParser(description="Run MIRand.")

    parser.add_argument('--input-struc', nargs='?', default='graph/karate.edgelist',
                        help='Input graph path for structure-layer')

    parser.add_argument('--input-attr', nargs='?', default='graph/karate.edgelist',
                        help='Input graph path for attributes-layer')

    parser.add_argument('--dataset', nargs='?', default='karate',
                        help='Input graph name for saving files')

    parser.add_argument('--output', nargs='?', default='emb/karate.emb',
                        help='Embeddings path')

    parser.add_argument('--dimensions', type=int, default=128,
                        help='Number of dimensions. Default is 128.')

    parser.add_argument('--walk-length', type=int, default=80,
                        help='Length of walk per source. Default is 80.')

    parser.add_argument('--num-walks', type=int, default=10,
                        help='Number of walks per source. Default is 10.')

    parser.add_argument('--window-size', type=int, default=10,
                        help='Context size for optimization. Default is 10.')

    parser.add_argument('--iter', default=1, type=int,
                        help='Number of epochs in SGD')

    parser.add_argument('--workers', type=int, default=8,
                        help='Number of parallel workers. Default is 8.')

    parser.add_argument('--p-struc', type=float, default=1,
                        help='Return hyperparameter for Structure-Layer. Default is 1.')

    parser.add_argument('--q-struc', type=float, default=1,
                        help='Inout hyperparameter for Structure-Layer. Default is 1.')

    parser.add_argument('--p-attr', type=float, default=1,
                        help='Return hyperparameter for Attribute-Layer. Default is 1.')

    parser.add_argument('--q-attr', type=float, default=1,
                        help='Inout hyperparameter for Attribute-Layer. Default is 1.')

    parser.add_argument('--weighted-struc', dest='weighted', action='store_true',
                        help='Boolean specifying (un)weighted Structure-Layer. Default is unweighted.')
    parser.add_argument('--unweighted-struc', dest='unweighted', action='store_false')
    parser.set_defaults(weighted_struc=False)

    parser.add_argument('--weighted-attr', dest='weighted', action='store_true',
                        help='Boolean specifying (un)weighted Attribute-Layer. Default is weighted.')
    parser.add_argument('--unweighted-attr', dest='unweighted', action='store_false')
    parser.set_defaults(weighted_attr=True)

    parser.add_argument('--directed-struc', dest='directed', action='store_true',
                        help='Structure-Layer Graph is (un)directed. Default is undirected.')
    parser.add_argument('--undirected-struc', dest='undirected', action='store_false')
    parser.set_defaults(directed_struc=False)

    parser.add_argument('--directed-attr', dest='directed', action='store_true',
                        help='Attribute-Layer Graph is (un)directed. Default is directed.')
    parser.add_argument('--undirected-attr', dest='undirected', action='store_false')
    parser.set_defaults(directed_attr=True)

    return parser.parse_args()

In [8]:
def read_graph():
    '''
    Reads the structure and attribute network in networkx.
    '''
    if args.weighted_attr:
        A = nx.read_edgelist(args.input_attr, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
    else:
        A = nx.read_edgelist(args.input_attr, nodetype=int, create_using=nx.DiGraph())
        for edge in A.edges():
            A[edge[0]][edge[1]]['weight'] = 1

    if args.weighted_struc:
        S = nx.read_edgelist(args.input_struc, nodetype=int, data=(('weight', float),), create_using=nx.DiGraph())
    else:
        S = nx.read_edgelist(args.input_struc, nodetype=int, create_using=nx.DiGraph())
        for edge in S.edges():
            S[edge[0]][edge[1]]['weight'] = 1

    if not args.directed_struc:
        S = S.to_undirected()

    if not args.directed_attr:
        A = A.to_undirected()

    return S, A

In [9]:
def generate_walks(G, start=False):
    #################################
    # Calculate transition probabilities for switching the levels
    print('\nStructure layer trans-prob evaluation started')
    G.get_level_transition_weight(0)
    print('\nAttribute layer trans-prob evaluation started')
    G.get_level_transition_weight(1)

    # print('\nalias 1 started')
    G.preprocess_transition_probs(0)
    # print('\nalias 2 started')
    G.preprocess_transition_probs(1)

    print('\nWalk simulation started')
    walks = G.simulate_walks(args.num_walks, args.walk_length)
    return walks

In [10]:
def learn_embeddings(G):
    walks = [map(str, walk) for walk in generate_walks(G, start=True)]
    model = Word2Vec(walks, size=args.dimensions, window=args.window_size, min_count=0, sg=1, workers=args.workers, iter=args.iter)
    model.save_word2vec_format(args.output)
    return

In [11]:
def main(args):
    print('Running MIRand for %s dataset\n' % args.dataset)
    nx_S, nx_A = read_graph()
    G = mirand.Graph(nx_S, nx_A, args.directed_struc, args.directed_attr, args.p_struc, args.q_struc, args.p_attr, args.q_attr)

    assert len(nx_S.nodes()) == len(nx_A.nodes())
    print("Number of nodes in the graph : %s" % len(nx_S.nodes()))
    print("Number of edges in the structure graph : %s" % len(nx_S.edges()))
    print("Number of edges in the attribute graph : %s" % len(nx_A.edges()))

    learn_embeddings(G)

In [13]:
nx_S,nx_A = read_graph()

NameError: name 'args' is not defined